# Note on titanic binary classification project

**Objective**: Here I attempt to build a simple neural network from scratch

** Step 1 **: Load libraries

In [5]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras as k

from sklearn import preprocessing

%matplotlib inline

** Step 2 **: Need to load data set from a csv file. Numpy's genfromtxt can be used, however, because ',' is present in string, use pandas's read_csv instead.

In [9]:
#load data sets
train_set = pd.read_csv('data/train.csv', delimiter=',', quotechar='"')
test_set = pd.read_csv('data/test.csv', delimiter=',', quotechar='"')

#have a peep at the training data
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
#data types in the training set
#train_set.dtypes

#missing data in 'Cabin'
print('Missing data in ''Cabin'' =', train_set['Cabin'].isnull().sum())

#total missing data
print('Missing data in each feature in training set:\n', train_set.isnull().sum())

#replacing missing values in 'Age' with median age
median_age = train_set['Age'].median()
train_set['Age'].fillna(median_age, inplace=True)
print('Median age =', median_age)
#print(train_set.isnull().sum())

#check missing values in test set
print('Missing data in each feature in test set:\n', test_set.isnull().sum())
#also replace mssing values in 'Age' with median age
median_age = test_set['Age'].median()
test_set['Age'].fillna(median_age, inplace=True)
print('Median age in test set =', median_age)

Missing data in Cabin = 687
Missing data in each feature in training set:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Median age = 28.0
Missing data in each feature in test set:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
Median age in test set = 27.0


The shape of the training set is correctly (891, 12), and that of the test set (418, 11). Note that the test set has one fewer input feature because we don't know the survival of each passenger.

Next, determine the number of training examples n_train to be 891 and  number of test examples n_test 418.

In [11]:
n_train = train_set.shape[0]
n_test = test_set.shape[0]
n_x = train_set.shape[1]-2 #passenger ID and survived

print("Number of training examples: n_train = " + str(n_train))
print("Number of testing examples: n_test = " + str(n_test))
print("Number of features for each example: n_x = " + str(n_x))

Number of training examples: n_train = 891
Number of testing examples: n_test = 418
Number of features for each example: n_x = 10


In [12]:
#Extract data from train_set
train_features = [train_set['Pclass'], train_set['Sex'], train_set['Age'], train_set['SibSp'], train_set['Parch'], train_set['Fare']]
X1 = pd.concat(train_features, axis=1)
#X1.head()
n_features = X1.shape[1]
print('Number of features to train = ', n_features)

#Extract data into X_train and Y_train, now numpy arrays -- not a good way!!!!
X_train = X1.iloc[:, :].values #matrix dimensions (n_train, n_x-4) excluded 'Name', 'Ticket', 'Cabin', 'Embarked'
Y_train = train_set.iloc[:, 1].values #vector dimension (n_train)
gender = preprocessing.LabelEncoder()
X_train[:, 1] = gender.fit_transform(X_train[:, 1])
print('X_train', X_train[:, :])

#Extract data from train_set
test_features = [test_set['Pclass'], test_set['Sex'], test_set['Age'], test_set['SibSp'], test_set['Parch'], test_set['Fare']]
X2 = pd.concat(test_features, axis=1)
X2.head()

X_test = X2.iloc[:, :].values
X_test[:, 1] = gender.fit_transform(X_test[:, 1])
print('X_test', X_test[0:5, :])

Number of features to train =  6
X_train [[3 1 22.0 1 0 7.25]
 [1 0 38.0 1 0 71.2833]
 [3 0 26.0 0 0 7.925]
 ...
 [3 0 28.0 1 2 23.45]
 [1 1 26.0 0 0 30.0]
 [3 1 32.0 0 0 7.75]]
X_test [[3 1 34.5 0 0 7.8292]
 [3 0 47.0 1 0 7.0]
 [2 1 62.0 0 0 9.6875]
 [3 1 27.0 0 0 8.6625]
 [3 0 22.0 1 1 12.2875]]


In [13]:
#normalizing inputs - appears to me so far using mean and variance 
sc = preprocessing.StandardScaler()
X_train = sc.fit_transform(X_train)
X_trainT = np.transpose(X_train)

X_test = sc.fit_transform(X_test)
X_testT = np.transpose(X_test)
print('X_train', X_train[:, :], X_trainT.shape)
print('X_test', X_test[:, :], X_testT.shape)

X_train [[ 0.82737724  0.73769513 -0.56573646  0.43279337 -0.47367361 -0.50244517]
 [-1.56610693 -1.35557354  0.66386103  0.43279337 -0.47367361  0.78684529]
 [ 0.82737724 -1.35557354 -0.25833709 -0.4745452  -0.47367361 -0.48885426]
 ...
 [ 0.82737724 -1.35557354 -0.1046374   0.43279337  2.00893337 -0.17626324]
 [-1.56610693  0.73769513 -0.25833709 -0.4745452  -0.47367361 -0.04438104]
 [ 0.82737724  0.73769513  0.20276197 -0.4745452  -0.47367361 -0.49237783]] (6, 891)
X_test [[ 0.87348191  0.75592895  0.38623105 -0.49947002 -0.4002477  -0.49781052]
 [ 0.87348191 -1.32287566  1.37137004  0.61699237 -0.4002477  -0.51265996]
 [-0.31581919  0.75592895  2.55353683 -0.49947002 -0.4002477  -0.46453181]
 ...
 [ 0.87348191  0.75592895  0.70147553 -0.49947002 -0.4002477  -0.50818292]
 [ 0.87348191  0.75592895 -0.20485235 -0.49947002 -0.4002477  -0.4938564 ]
 [ 0.87348191  0.75592895 -0.20485235  0.61699237  0.61989583 -0.23762123]] (6, 418)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Ideally, we can extract the training examples $X$ and output labels $Y$ from the training set and implement the different functions as shown below. However, there are different data types in the training set.

In [14]:
# attempt to use Keras to train a fully connected network
model = k.models.Sequential()

#add input layer and the first hidden layer
model.add(k.layers.Dense(5, kernel_initializer='uniform', input_shape=(n_features,), activation='relu'))

#add second hidden layer
model.add(k.layers.Dense(5, kernel_initializer='uniform', activation='relu'))

#add output layer
model.add(k.layers.Dense(1, kernel_initializer='uniform', activation='sigmoid'))

#compile network
optimizer = k.optimizers.Adam(lr=0.005)
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])

#run network
model.fit(X_train, Y_train, batch_size=20, epochs=10)

Epoch 1/10
891/891 [==============================] - 2s 2ms/step - loss: 0.6789 - acc: 0.6094
Epoch 2/10
891/891 [==============================] - 0s 155us/step - loss: 0.5587 - acc: 0.7273
Epoch 3/10
891/891 [==============================] - 0s 155us/step - loss: 0.4761 - acc: 0.7969
Epoch 4/10
891/891 [==============================] - 0s 157us/step - loss: 0.4616 - acc: 0.7980
Epoch 5/10
891/891 [==============================] - 0s 159us/step - loss: 0.4575 - acc: 0.8013
Epoch 6/10
891/891 [==============================] - 0s 157us/step - loss: 0.4461 - acc: 0.8047
Epoch 7/10
891/891 [==============================] - 0s 156us/step - loss: 0.4400 - acc: 0.8036
Epoch 8/10
891/891 [==============================] - 0s 153us/step - loss: 0.4365 - acc: 0.8025
Epoch 9/10
891/891 [==============================] - 0s 161us/step - loss: 0.4310 - acc: 0.8081
Epoch 10/10
891/891 [==============================] - 0s 206us/step - loss: 0.4292 - acc: 0.8070


In [15]:
Y_pred = model.predict(X_test, batch_size=100)
Y_pred = (Y_pred > 0.5).astype(int)
print(Y_pred.shape)
Y = pd.DataFrame({'Survived' : Y_pred[:, 0]})
submission = pd.concat((test_set['PassengerId'], Y), axis=1)
submission.head(100)
print('Number of passengers predicted to survive =', Y.sum())
submission.to_csv('data/submission_AR.csv', index=False)

(418, 1)
Number of passengers predicted to survive = Survived    149
dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  
